In [1]:
import sys
sys.executable

'/root/home/envforir/bin/python'

In [2]:
# # 데몬 인스턴스 만들기 
import os
import json
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT

# es_server = Popen(['/root/home/elasticsearch-8.8.0/bin/elasticsearch'],
#                   stdout=PIPE, 
#                   stderr=STDOUT,
#                   preexec_fn=lambda: os.setuid(1))

# import time
# time.sleep(60)

In [3]:
# 데몬 구동 확인 -> password 설정 
!ps -ef | grep elasticsearch

iruser    575561       1 99 01:01 pts/12   00:00:51 /home/iruser/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j2.formatMsgNoLookups=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=org.elasticsearch.preallocate -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-14538001589106802100 -XX:+HeapDumpOnOutOfMemoryError -XX:+ExitOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,level,pid,tags:filecount=32,filesize=64m -Xms30720m -Xmx30720m -XX:MaxDirectMemorySize=16106127360 -XX:InitiatingHeapOccupancyPercent=30

In [4]:
es_username = 'elastic'
es_password = '8YoPc7sP_W-uBUDkXV73'

# ElasticSearch Client 생성 
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/data/ephemeral/home/upstage-ai-final-ir2/HM/elasticsearch-8.8.0/config/certs/http_ca.crt", verify_certs=False)

# ElasticSearch Client 정보 확인 
es.info()

/root/home/envforir/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'name': 'instance-8909', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'r4Ih9u1sTfKbhPDz61kqLQ', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화(한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")   
# 이 모델을 사용하여 doc['content']를 임베딩 

# Sentence Transformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)

# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i: i+batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)    # doc의 "content" 부분을 임베딩 
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings

### 인덱스 설정 함수: Sparse Retrieve, Dense Retrieve를 위한 인덱스 생성과 관리
- Dense Retrieve: 인덱스 맵핑 뿐만 아니라 밀집 벡터 필드를 추가해야

In [6]:
# 새로운 index 생성 
def create_es_index(index, settings, mappings):
    if es.indices.exists(index=index):   # 만약 인덱스가 이미 존재한다면 
        es.indices.delete(index=index)   # 새로운 것으로 설정하기 위해 삭제 
    es.indices.create(index=index, settings=settings, mappings=mappings)  # 지정된 settings로 새로운 인덱스 생성 

# 지정된 인덱스 삭제 
def delete_es_index(index):
    es.indices.delete(index=index)
    
# ElasticSearch 헬퍼 함수를 사용하여 대량 인덱싱 수행 
def bulk_add(index, docs):
    actions = [{'_index': index, '_source': doc} for doc in docs]
    return helpers.bulk(es, actions)

### 검색 함수
- Sparse Retrieve: 텍스트 기반 검색 
- Dense Retrieve: 벡터 유사도 기반 검색

In [7]:
# 역색인을 이용한 검색 
def sparse_retrieve(query_str, size):
    query = { "match": {"content": {"query": query_str}}}
    return es.search(index='test', query=query, size=size, sort='_score')
#>> query_str: 검색어
#>> size: 검색 결과 개수 
#>> 'test' 인덱스에서 정의된 쿼리를 실행 -> 검색 결과를 size개 만큼 반환 


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    query_embedding = get_embedding([query_str])[0]   # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기 
    knn = {"field": "embeddings",                     # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정 
           "query_vector": query_embedding.tolist(),
           "k": size,    # k: 검색할 최근접 이웃의 개수 
           "num_candidates": 100}  # KNN 검색에 사용할 후보 문서 개수 
    return es.search(index='test', knn=knn)           # 지정된 인덱스에서 벡터 유사도 검색 수행 

In [8]:
# dense_retrieve_2: sparse_retrieve 함수로 필터링된 문서들에 대해서만 KNN 검색
def dense_retrieve_2(query_str, documents, size):
    query_embedding = get_embedding([query_str][0])
    document_ids = [doc['_id'] for doc in documents]
    # document_embeddings = [doc['_source']['embeddings'] for doc in documents]
    
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": len(documents)
    }
    
    # 미리 필터링된 문서들에 대해서만 'test' 인덱스에서 KNN 검색 수행 
    body = {
        "knn": knn,
        "_source": {"includes": ["content", "embeddings"]},
        "query": {"ids": {"values": document_ids}}
    }
    
    return es.search(index='test', body=body)
    

In [43]:
# def dense_retrieve_2(query_str, documents, size):
#     query_embedding = get_embedding([query_str])[0]  # [0]을 추가하여 리스트에서 벡터를 추출
    
#     body = {
#         "size": size,
#         "query": {
#             "script_score": {
#                 "query": {
#                     "terms": {
#                         "_id": [doc['_id'] for doc in documents]
#                     }
#                 },
#                 "script": {
#                     "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
#                     "params": {
#                         "query_vector": query_embedding.tolist()
#                     }
#                 }
#             }
#         }
#     }
    
#     return es.search(index='test', body=body)

### 설정

In [9]:
# 색인을 위한 settings 설정 
settings = { "analysis": {"analyzer": {"nori": {"type": "custom",  #>> custom: 사용자 정의 분석기임을 명시 
                                                "tokenizer": "nori_tokenizer",
                                                "decompound_mode": "mixed",  # 복합 명사를 분해하는 방식을 설정. mixed: 복합명사를 분해하되, 분해된 토큰과 원본 토큰을 모두 저장 
                                                "filter": ["nori_posfilter"]}},
                          "filter": {"nori_posfilter":{"type": "nori_part_of_speech",
                                                       "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]}}}  # 제거할 품사 태그를 지정 
            }

# 색인을 위한 mapping 설정(역색인 필드, 임베딩 필드 모두 설정)
mappings = { "properties": {"content": {"type": "text",
                                        "analyzer": "nori"},
                            "embeddings": {"type": "dense_vector",
                                           "dims": 768,
                                           "index": True,
                                           "similarity": "l2_norm"}}  # 벡터 간 유사도 계산 방식: L2 norm 
            }


In [10]:
# 위에서 설정한 내용으로 'test' 인덱스 생성 
create_es_index("test", settings, mappings)

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 임베딩: 문서의 content 필드

In [11]:
# 문서의 content 필드에 대한 임베딩 생성 
index_docs = []
with open("/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)


batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/c

(4272, [])


In [12]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"

### Sparse Retrieval 예제 

In [13]:
# 역색인을 사용하는 검색 예제 
search_result_retrieve = sparse_retrieve(test_query, 10)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']['content'])

score: 33.68643 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 18.915426 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 나이는 7억 5천만 년으로 추정됩니다. 금성은 지구와 매우 비슷한 크기와 구성을 가지고 있으며, 약 90% 이상이 이산화탄소로 이루어져 있습니다. 이 행성은 매우 뜨거운 온도와 압력을 가지고 있어서 인간이 살 수 있는 환경이 아닙니다. 금성의 대기는 두꺼워서 태양의 열을 가두고 있어서 행성의 표면은 평균 온도가 약 450도로 매우 뜨거운 상태입니다. 또한, 금성은 자전 속도가 매우 빠르기 때문에 하루가 지구의 약 243일과 같습니다. 이러한 특징들로 인해 금성은 우리 태양계에서 가장 가혹한 환경을 가진 행성 중 하나로 알려져 있습니다.
score: 18.728905 source: 메릴랜드 Space Grant Observatory는 볼티모어에 위치해 있습니다. 학생들은 이 망원경을 방문하여 별, 행성, 달에 대해 배웠습니다. 그들은 아래와 같은 정보를 기록했습니다. 첫째, 별 패턴은 그대로 유지되지만, 하늘에서의 위치는 변하는 것처럼 보입니다. 둘째, 태양, 행성, 달은 하늘에서 움직이는 것처럼 보입니다. 셋째, 켄타우루스자리의 프록시마 성은 우리 태양계에서 가장 가까운 별입니다. 넷째, 북극성은 소 북두칠성이라 불리는 별 패턴의 일부입니다. 그렇다면 태양이 매일 하늘을 가로질러 움직이는 것처럼 보이는 이유는 무엇일까요? 지구가 자전축을 중심으로 회전하기

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Dense Retrieval 예제

In [15]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 10)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.0047387853 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 0.004689848 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.
score: 0.0042649144 source: 소행성대가 위치한 곳에는 행성이 없는 이유는 목성과의 공명으로 인해 물질이 모이는 것을 방해하여 행성을 형성하지 못했기 때문입니다. 소행성대는 태양계 내부 행성들과 외부 행성들 사이에 위치한 영역으로, 많은 소행성들이 모여있는 지역입니다. 하지만 목성과의 공명 현상으로 인해 소행성들이 목성의 중력에 영향을 받아 행성을 형성하지 못하고 분산되거나 파괴되는 경우가 많습니다. 따라서 소행성대는 행성이 형성되기 어려운 환경이라고 할 수 있습니다. 이러한 이유로 소행성대에는 행성이 없는 것입니다.
score: 0.0038002816 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve_2(test_query, search_result_retrieve['hits']['hits'], 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 1.0047388 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 1.0046898 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.
score: 1.004265 source: 소행성대가 위치한 곳에는 행성이 없는 이유는 목성과의 공명으로 인해 물질이 모이는 것을 방해하여 행성을 형성하지 못했기 때문입니다. 소행성대는 태양계 내부 행성들과 외부 행성들 사이에 위치한 영역으로, 많은 소행성들이 모여있는 지역입니다. 하지만 목성과의 공명 현상으로 인해 소행성들이 목성의 중력에 영향을 받아 행성을 형성하지 못하고 분산되거나 파괴되는 경우가 많습니다. 따라서 소행성대는 행성이 형성되기 어려운 환경이라고 할 수 있습니다. 이러한 이유로 소행성대에는 행성이 없는 것입니다.
score: 1.0 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 나이는 7억 5천만 년으로 추정됩

/tmp/ipykernel_575239/2372825389.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index='test', body=body)
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# RAG 구현
준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [19]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = "sk-proj-FCdLR9ZMWFm8vjGzk3p3T3BlbkFJi3Dmulq4Tl3lOhZpHMoK"
#os.environ["OPENAI_API_KEY"] = "Your API Key"

client = OpenAI()

# 사용할 모델을 설정
llm_model = "gpt-4-turbo"


# 프롬프트 수정

In [20]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트 
# : 질의을 분석하고 standalone_query를 생성 
# 최종 답변 생성을 위한 프롬프트 
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다. 
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
# : standalone_query가 추출된 이후 그 standalone_query로 검색 요청 
# -> 검색결과로 나온 문서들을 reference로 전달
# -> 그 reference와 함께 질의에 대한 답을 생성 
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다. 
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "사용자 메세지 내역으로부터 검색에 사용할 적합한 최종 질의"
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]
#>> 사용함수는 'search' 
#>> 'search' 함수에는 파라미터가 필요: 'standalone_query'
# 

In [21]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 1. 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 2. 검색이 필요한 경우(Function Calling이 유도된 경우) 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)  # json.loads: str -> json
        standalone_query = function_args.get("standalone_query")

        # sparse_retrieve로 10개 추출
        sparse_results = sparse_retrieve(standalone_query, 10)
        
        # dense_retrieve로 3개 추출
        dense_results = dense_retrieve_2(standalone_query, sparse_results['hits']['hits'], 3)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i,rst in enumerate(dense_results['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 3. 검색이 필요하지 않은 경우(과학상식에 관한 질문이 아니라서 Function Calling이 유도되지 않음) 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response



In [22]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 5:
            #   break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [27]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/data/ephemeral/home/upstage-ai-final-ir2/HM/data/eval.jsonl", "sample_submission_3.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]


/home/iruser/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/iruser/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Answer: 나무의 분류를 조사하기 위해서는 다음과 같은 방법을 사용할 수 있습니다:

1. **현장 조사**: 다양한 지역을 방문하여 나무의 생육 환경, 크기, 잎의 모양과 배열, 꽃과 열매의 특성 등을 관찰합니다.

2. **도감 및 참고자료 활용**: 나무에 대한 도감이나 식물학 관련 서적을 참고하여 관찰한 나무의 특징과 비교하며 분류합니다.

3. **전문가 상담**: 식물학자나 수목원의 전문가에게 조언을 구하거나 함께 현장 조사를 진행할 수 있습니다.

4. **DNA 분석**: 최근에는 유전자 분석을 통해 나무의 종을 정확하게 분류하는 방법도 사용됩니다.

이러한 방법들을 통해 나무의 종류를 정확하게 분류하고 그 특성을 이해할 수 있습니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]


/home/iruser/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/iruser/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Answer: 제공된 정보에는 각 나라의 공교육 지출 현황에 대한 구체적인 데이터가 포함되어 있지 않습니다. 따라서 이에 대한 자세한 답변을 제공할 수 없습니다. 다만, 전반적으로 세계적으로 공교육에 대한 지출은 GDP의 약 4%를 차지하고 있다는 점을 언급할 수 있습니다. 이는 국가들이 교육에 상당한 자원을 투자하고 있음을 나타냅니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]


...

Answer: 기억 상실증은 다양한 원인으로 발생할 수 있습니다. 주요 원인으로는 뇌 손상, 알츠하이머병, 기타 치매 질환, 뇌졸중, 심한 스트레스나 정신적 충격, 약물의 부작용, 감염, 영양 결핍 등이 있습니다. 이러한 원인들은 뇌의 기억을 담당하는 부위에 영향을 미쳐 기억력에 장애를 일으킬 수 있습니다.

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]


Traceback (most recent call last):
  File "/home/iruser/miniconda3/lib/python3.12/site-packages/httpx/_transports/default.py", line 69, in map_httpcore_exceptions
    yield
  File "/home/iruser/miniconda3/lib/python3.12/site-packages/httpx/_transports/default.py", line 233, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iruser/miniconda3/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 216, in handle_request
    raise exc from None
  File "/home/iruser/miniconda3/lib/python3.12/site-packages/httpcore/_sync/connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iruser/miniconda3/lib/python3.12/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    return self._connection.handle_request(request)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iruser/miniconda3/lib/pytho

Answer: 

Test 4
Question: [{'role': 'user', 'content': 'Dmitri Ivanovsky가 누구야?'}]


/home/iruser/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/iruser/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


BadRequestError: BadRequestError(400, 'x_content_parse_exception', 'Failed to build [knn] after last required field arrived')